# US Soil Information Analysis Demo

This notebook demonstrates how to analyze soil data for a specific geographic region using the USDA Soil Data Access (SDA) API. The workflow includes:

1. **Data Visualization**: Creating interactive maps with satellite imagery
2. **Spatial Analysis**: Working with polygon geometries and coordinate systems
3. **API Integration**: Querying USDA soil databases
4. **Data Processing**: Retrieving and analyzing soil properties

## Overview
We'll analyze soil characteristics for a polygon area near Lawrence, Kansas, including soil taxonomy, drainage, pH, organic matter, and particle size distribution.

## Setup and Dependencies

First, we'll install the required Python packages for geospatial analysis, data manipulation, and API interactions.

In [1]:
# Install required packages for geospatial analysis and data processing
# -q flag suppresses verbose output for cleaner display
!pip -q install aiohttp      # Asynchronous HTTP client for efficient API calls
!pip -q install folium       # Interactive mapping library
!pip -q install shapely      # Geometric operations and spatial analysis
!pip -q install geopandas    # Geospatial data manipulation

!pip -q install requests     # HTTP library for API requests!jupyter trust ./example_1.ipynb

# Trust the notebook to allow interactive widgets

In [2]:
# Core libraries for geospatial analysis and data processing
import folium                          # Interactive mapping and visualization
from typing import List, Dict, Any     # Type hints for better code documentation
import pandas as pd                    # Data manipulation and analysis
import random                          # Random color generation for map styling
import requests                        # HTTP requests for API calls
from shapely import wkt               # Well-Known Text format handling
import geopandas as gpd               # Geospatial data operations

## Import Required Libraries

We'll use several key libraries:
- **folium**: For interactive mapping and visualization
- **shapely**: For geometric operations and spatial analysis
- **geopandas**: For geospatial data manipulation
- **pandas**: For data processing and analysis
- **requests**: For API calls to USDA services
- **aiohttp**: For asynchronous HTTP requests (performance optimization)

In [3]:
# Define our study area as a polygon in GeoJSON format
# This polygon represents a specific agricultural area near Lawrence, Kansas
# Coordinates are in WGS84 format [longitude, latitude]
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},  # Empty properties - we'll add soil data later
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [   # Outer ring of the polygon (clockwise order)
                        [-95.320923, 39.587342],  # Northwestern corner
                        [-95.32088, 39.585573],   # Start moving southeast
                        [-95.312769, 39.580893],  # Southeastern corner
                        [-95.311654, 39.58119],   # Start moving north
                        [-95.311589, 39.582034],  # Continue northward
                        [-95.311611, 39.584283],  # Moving to northwest
                        [-95.311632, 39.587193],  # Northwestern edge
                        [-95.311868, 39.587408],  # Western boundary
                        [-95.314829, 39.587408],  # Northern boundary
                        [-95.319786, 39.587442],  # Back to start area
                        [-95.320923, 39.587342],  # Close the polygon
                    ]
                ],
            },
        }

    ],}

## Define Study Area

We'll define our area of interest as a polygon in GeoJSON format. This polygon represents a specific location near Lawrence, Kansas. The coordinates are in WGS84 (latitude/longitude) format.

In [4]:
# Create an interactive map with satellite imagery background
# Step 1: Calculate the center point of our polygon for map centering
coords = geojson["features"][0]["geometry"]["coordinates"][0]
center_lat = sum([coord[1] for coord in coords]) / len(coords)  # Average latitude
center_lon = sum([coord[0] for coord in coords]) / len(coords)  # Average longitude

# Step 2: Create a Folium map with USGS satellite imagery tiles
# Using high-resolution satellite imagery for detailed terrain visualization
base_map = folium.Map(
    location=[center_lat, center_lon],  # Center the map on our study area
    zoom_start=15,                      # High zoom level for detailed view
    # USGS National Map imagery tiles for high-quality satellite view
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}",
    attr="USGS",                        # Attribution for the imagery
)

# Step 3: Add our study area polygon as an overlay on the map
# This will show the exact boundaries of our area of interest
folium.GeoJson(
    geojson,
    style_function=lambda x: {
        'fillColor': 'red',      # Red fill for visibility
        'color': 'red',          # Red border
        'weight': 2,             # Border thickness
        'fillOpacity': 0.3       # Semi-transparent fill
    }
).add_to(base_map)

# Display the interactive map
base_map

## Visualize the Study Area

Let's create an interactive map showing our study area overlaid on satellite imagery. We'll use USGS satellite tiles for a high-resolution view of the terrain.

In [5]:
# Convert GeoJSON polygon to Well-Known Text (WKT) format
# The USDA Soil Data Access API requires geometry in WKT format
from shapely.geometry import shape


# Extract the geometry from GeoJSON and convert to WKT stringprint(f"Study area in WKT format: {study_area_wkt}")
study_area_wkt = shape(geojson["features"][0]["geometry"]).wkt

## Convert to Well-Known Text (WKT) Format

The USDA Soil Data Access API requires geometry in WKT format. We'll convert our GeoJSON polygon to WKT, which is a standard format for representing geometric objects.

In [6]:
# USDA Soil Data Access (SDA) API endpoint
# This RESTful service allows SQL queries against the SSURGO soil database
sda_api_url = "https://sdmdataaccess.nrcs.usda.gov/Tabular/SDMTabularService/post.rest"

## Connect to USDA Soil Data Access API

The USDA provides the Soil Data Access (SDA) web service for querying soil survey data. This RESTful API allows us to execute SQL queries against the soil database.

In [7]:
# Query the USDA database to find all soil map units (MUKEYs) that intersect our study area
# MUKEY = Map Unit Key: unique identifier for each soil mapping unit
mukey_query = f"SELECT mukey FROM SDA_Get_Mukey_from_intersection_with_WktWgs84('{study_area_wkt}')"

# Send POST request to the SDA API
mukey_response = requests.post(
    sda_api_url,
    json={"format": "JSON", "query": mukey_query},
    headers={"Content-Type": "application/json"},
)

# Extract MUKEYs from the API response
# The response contains a "Table" with rows of data
soil_map_unit_keys: List[str] = [row[0] for row in mukey_response.json()["Table"]]

print(f"Found {len(soil_map_unit_keys)} soil map units in study area:")
print(soil_map_unit_keys)

Found 3 soil map units in study area:
['1475356', '1475358', '2505122']


## Get Map Unit Keys (MUKEYs)

First, we need to identify which soil map units intersect with our study area. The `mukey` (Map Unit Key) is a unique identifier for each soil map unit in the database.

In [8]:
# Comprehensive SQL query to retrieve detailed soil properties for each map unit
# This complex query joins multiple SSURGO database tables to get complete soil information

soil_properties_query = f"""
WITH RankedSoils AS (
    SELECT 
        m.mukey,                    -- Map unit key (unique identifier)
        m.muname,                   -- Map unit name (descriptive name)
        c.taxorder,                 -- Soil taxonomic order (highest classification level)
        c.taxsuborder,              -- Soil taxonomic suborder 
        c.drainagecl,               -- Drainage class (e.g., well drained, poorly drained)
        c.taxreaction,              -- Soil reaction (pH category)
        c.taxclname,                -- Taxonomic class name
        h.ph1to1h2o_r,              -- pH in water (1:1 ratio) - representative value
        h.om_r,                     -- Organic matter content (%) - representative value
        h.sandtotal_r,              -- Total sand percentage - representative value
        h.silttotal_r,              -- Total silt percentage - representative value
        h.claytotal_r,              -- Total clay percentage - representative value
        tg.texture,                 -- Soil texture class (e.g., loam, clay loam)
        -- Ranking system to get the most representative soil data:
        -- 1. Highest component percentage (most dominant soil in the map unit)
        -- 2. Shallowest horizon depth (surface soil properties)
        ROW_NUMBER() OVER (
            PARTITION BY m.mukey 
            ORDER BY c.comppct_r DESC, h.hzdept_r ASC
        ) AS soil_rank
    FROM mapunit m                  -- Map units table
    INNER JOIN component c ON m.mukey = c.mukey           -- Soil components in each map unit
    INNER JOIN chorizon h ON h.cokey = c.cokey            -- Soil horizons for each component
    INNER JOIN chtexturegrp tg ON tg.chkey = h.chkey      -- Texture information for each horizon
    WHERE m.mukey IN ({', '.join(soil_map_unit_keys)})    -- Filter to our study area map units
        AND c.majcompflag = 'Yes'                         -- Only major components (primary soils)
)
SELECT 
    mukey, muname, taxorder, taxsuborder, drainagecl, taxreaction, taxclname,
    ph1to1h2o_r, om_r, sandtotal_r, silttotal_r, claytotal_r, texture
FROM RankedSoils
WHERE soil_rank = 1  -- Get only the most representative soil for each map unit
"""

# Execute the query and get soil properties data
soil_properties_response = requests.post(
    url=sda_api_url,
    json={"format": "JSON", "query": soil_properties_query},
    headers={"Content-Type": "application/json"},
)

# Process the response and create structured soil data
print("Raw API response structure:")
print(soil_properties_response.json().keys())

# Convert the tabular response to a list of dictionaries for easier processing
soil_properties_data: List[Dict[str, Any]] = []
for row in soil_properties_response.json()["Table"]:
    soil_properties_data.append({
        "mukey": row[0],            # Map unit key
        "muname": row[1],           # Map unit name
        "taxorder": row[2],         # Taxonomic order
        "taxsuborder": row[3],      # Taxonomic suborder
        "drainagecl": row[4],       # Drainage class
        "taxreaction": row[5],      # Tax reaction
        "taxclname": row[6],        # Taxonomic class name
        "ph1to1h2o_r": row[7],      # pH in water
        "om_r": row[8],             # Organic matter %
        "sandtotal_r": row[9],      # Sand %
        "silttotal_r": row[10],     # Silt %
        "claytotal_r": row[11],     # Clay %
        "texture": row[12]          # Texture class
    })

# Create a pandas DataFrame for easier data analysis and visualization
soil_properties_df = pd.DataFrame(soil_properties_data)

print(f"\nRetrieved soil properties for {len(soil_properties_df)} map units:")
print(f"Columns: {list(soil_properties_df.columns)}")

# Display the soil properties data
soil_properties_df

Raw API response structure:
dict_keys(['Table'])

Retrieved soil properties for 3 map units:
Columns: ['mukey', 'muname', 'taxorder', 'taxsuborder', 'drainagecl', 'taxreaction', 'taxclname', 'ph1to1h2o_r', 'om_r', 'sandtotal_r', 'silttotal_r', 'claytotal_r', 'texture']


,mukey,muname,taxorder,taxsuborder,drainagecl,taxreaction,taxclname,ph1to1h2o_r,om_r,sandtotal_r,silttotal_r,claytotal_r,texture
0,1475356,"Grundy silty clay loam, 1 to 3 percent slopes",Mollisols,Udolls,Somewhat poorly drained,not used,"Fine, smectitic, mesic Aquertic Argiudolls",6.1,3.7,4,65,31,SICL
1,1475358,"Grundy silty clay loam, 3 to 7 percent slopes,...",Mollisols,Udolls,Somewhat poorly drained,not used,"Fine, smectitic, mesic Aquertic Argiudolls",6.1,3,3,65,32,SICL
2,2505122,"Pawnee clay loam, 4 to 8 percent slopes, eroded",Mollisols,Udolls,Moderately well drained,not used,"Fine, smectitic, mesic Oxyaquic Vertic Argiudolls",5.6,2,23,46,31,CL


## Retrieve Detailed Soil Properties

Now we'll query comprehensive soil information for each map unit. This complex SQL query joins multiple tables to get:

- **Taxonomic information**: taxorder, taxsuborder, taxclname
- **Physical properties**: drainage class, texture, particle sizes (sand, silt, clay)
- **Chemical properties**: pH, organic matter content
- **Map unit details**: name and key

The query uses a ranking system to get the most representative soil component and horizon for each map unit.

In [9]:
# Import libraries for asynchronous HTTP requests (performance optimization)
import aiohttp
import asyncio

# Dictionary to store polygon geometries for each map unit
soil_polygons_dict: Dict[str, Any] = {}

# Asynchronous function to fetch polygon data for a single map unit
async def fetch_soil_polygon_data(session, mukey):
    """
    Fetch polygon geometry data for a specific map unit key (MUKEY)
    
    Args:
        session: aiohttp session for making HTTP requests
        mukey: Map unit key to fetch polygon data for
    
    Returns:
        tuple: (mukey, polygon_data) - the map unit key and its polygon data
    """
    polygon_query = f"""SELECT * FROM SDA_Get_MupolygonWktWgs84_from_Mukey({mukey})"""
    
    async with session.post(
        sda_api_url, 
        json={"format": "JSON", "query": polygon_query}, 
        headers={"Content-Type": "application/json"}
    ) as response:
        return mukey, await response.json()

# Main asynchronous function to fetch all polygon data concurrently
async def fetch_all_soil_polygons():
    """
    Fetch polygon data for all map units simultaneously using async requests
    This is much faster than making sequential requests
    """
    async with aiohttp.ClientSession() as session:
        # Create tasks for all map units
        polygon_fetch_tasks = [
            fetch_soil_polygon_data(session, mukey) 
            for mukey in soil_map_unit_keys
        ]
        
        # Execute all requests concurrently and wait for completion
        polygon_results = await asyncio.gather(*polygon_fetch_tasks)
        
        # Process results and store in dictionary
        for mukey, polygon_data in polygon_results:
            polygon_wkt_list = polygon_data['Table']
            if len(polygon_wkt_list) > 0:
                soil_polygons_dict[mukey] = polygon_wkt_list
                print(f"Retrieved {len(polygon_wkt_list)} polygons for MUKEY {mukey}")

# Execute the async function to fetch all polygon data
print("Fetching polygon geometries for all soil map units...")
await fetch_all_soil_polygons()

print(f"\nSuccessfully retrieved polygon data for {len(soil_polygons_dict)} map units:")
print(list(soil_polygons_dict.keys()))

Fetching polygon geometries for all soil map units...
Retrieved 118 polygons for MUKEY 1475356
Retrieved 31 polygons for MUKEY 1475358
Retrieved 158 polygons for MUKEY 2505122

Successfully retrieved polygon data for 3 map units:
['1475356', '1475358', '2505122']
Retrieved 118 polygons for MUKEY 1475356
Retrieved 31 polygons for MUKEY 1475358
Retrieved 158 polygons for MUKEY 2505122

Successfully retrieved polygon data for 3 map units:
['1475356', '1475358', '2505122']


## Get Soil Map Unit Polygons

For spatial visualization, we need the actual geometric boundaries of each soil map unit. We'll use asynchronous requests to efficiently retrieve polygon data for all map units simultaneously.

In [10]:
# Advanced spatial analysis and visualization of soil map units
# This section creates an interactive map showing soil boundaries and calculates areas

# List to store processed soil features with geometric and area information
processed_soil_features = []

print("Processing soil map unit polygons and calculating intersections...")

# Process each soil map unit and its associated polygons
for mukey, polygon_wkt_list in soil_polygons_dict.items():
    map_features = []  # Features for this specific map unit
    
    print(f"Processing Map Unit {mukey}:")
    
    # Process each polygon associated with this map unit
    for i, wkt_polygon_data in enumerate(polygon_wkt_list):
        # Convert WKT string to Shapely geometry object
        soil_polygon_geometry = wkt.loads(wkt_polygon_data[0])
        
        # Check if this soil polygon intersects with our study area
        if soil_polygon_geometry.intersects(wkt.loads(study_area_wkt)):
            print(f"  - Polygon {i+1}: Intersects with study area")
            
            # Calculate the intersection geometry (only the overlapping part)
            intersection_geometry = soil_polygon_geometry.intersection(wkt.loads(study_area_wkt))
            
            # Area calculation using proper coordinate system transformation
            # Step 1: Create GeoDataFrame with the intersection geometry in WGS84
            intersection_gdf = gpd.GeoDataFrame(
                {'geometry': [intersection_geometry]}, 
                crs='EPSG:4326'  # WGS84 coordinate reference system
            )
            
            # Step 2: Transform to UTM Zone 15N for accurate area calculation
            # UTM provides measurements in meters, allowing precise area calculation
            intersection_gdf_utm = intersection_gdf.to_crs('EPSG:32615')
            
            # Step 3: Calculate area in square meters and convert to acres
            area_square_meters = intersection_gdf_utm.area[0]
            area_acres = area_square_meters / 4046.8564224  # 1 acre = 4046.8564224 m²
            
            print(f"    Area: {area_acres:.3f} acres ({area_square_meters:.1f} m²)")
            
            # Create GeoJSON feature for map visualization
            map_feature = {
                "type": "Feature",
                "properties": {
                    'mukey': mukey,
                    'area_acres': round(area_acres, 3)
                },
                "geometry": intersection_geometry.__geo_interface__
            }
            map_features.append(map_feature)
            
            # Store processed feature data for analysis
            processed_soil_features.append({
                "mukey": mukey,
                "area_acres": area_acres,
                "area_square_meters": area_square_meters,
                "geometry": intersection_geometry.__geo_interface__
            })
        else:
            print(f"  - Polygon {i+1}: No intersection with study area")
    
    # Add this map unit's features to the interactive map if any intersections exist
    if map_features:
        # Create GeoJSON collection for this map unit
        map_unit_geojson = {
            "type": "FeatureCollection",
            "features": map_features
        }
        
        # Generate a random color for this soil type
        soil_type_color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
        
        # Add to the map with custom styling
        folium.GeoJson(
            map_unit_geojson, 
            style_function=lambda feature, color=soil_type_color: {
                'fillColor': color,
                'color': color,
                'weight': 2,
                'fillOpacity': 0.6
            },
            popup=folium.Popup(
                f"Map Unit: {mukey}<br>Area: {map_features[0]['properties']['area_acres']} acres",
                parse_html=True
            )
        ).add_to(base_map)
        
        print(f"  Added {len(map_features)} features to map with color {soil_type_color}")

print(f"Visualization complete!")
print(f"Total soil features processed: {len(processed_soil_features)}")
print(f"Total area analyzed: {sum(f['area_acres'] for f in processed_soil_features):.3f} acres")

# Display the enhanced interactive map
base_map

Processing soil map unit polygons and calculating intersections...
Processing Map Unit 1475356:
  - Polygon 1: No intersection with study area
  - Polygon 2: No intersection with study area
  - Polygon 3: No intersection with study area
  - Polygon 4: No intersection with study area
  - Polygon 5: No intersection with study area
  - Polygon 6: No intersection with study area
  - Polygon 7: No intersection with study area
  - Polygon 8: No intersection with study area
  - Polygon 9: No intersection with study area
  - Polygon 10: No intersection with study area
  - Polygon 11: No intersection with study area
  - Polygon 12: No intersection with study area
  - Polygon 13: No intersection with study area
  - Polygon 14: No intersection with study area
  - Polygon 15: No intersection with study area
  - Polygon 16: No intersection with study area
  - Polygon 17: No intersection with study area
  - Polygon 18: No intersection with study area
  - Polygon 19: No intersection with study area
 

## Visualize Soil Map Units

Now we'll create a comprehensive visualization showing:
1. **Intersecting polygons**: Only the portions of soil map units that overlap with our study area
2. **Area calculations**: Convert geometries to UTM coordinates for accurate area measurements in acres
3. **Color coding**: Each soil type gets a unique color for easy identification

The visualization includes both the spatial representation and area calculations for further analysis.

In [11]:
# Display all processed soil features with their spatial and area information
# Each feature represents a portion of a soil map unit that intersects our study area
print(f"Processed soil features summary:")
print(f"Number of features: {len(processed_soil_features)}")
print(f"Total area covered: {sum(f['area_acres'] for f in processed_soil_features):.3f} acres")
print("\nDetailed feature data:")
processed_soil_features

Processed soil features summary:
Number of features: 3
Total area covered: 97.338 acres

Detailed feature data:


[{'mukey': '1475356',
  'area_acres': np.float64(56.96062297117019),
  'area_square_meters': np.float64(230511.46289478504),
  'geometry': {'type': 'Polygon',
   'coordinates': (((-95.3165186977073, 39.5872314566935),
     (-95.3164141878656, 39.5869765641111),
     (-95.3161397123363, 39.5868028394951),
     (-95.3159767702874, 39.5867626851803),
     (-95.3155682023138, 39.5867283262281),
     (-95.3153789062697, 39.5866175307974),
     (-95.315331305141, 39.5863094072921),
     (-95.3153483058068, 39.5861274595656),
     (-95.3154361140143, 39.5859128820583),
     (-95.3154331409465, 39.5857308250711),
     (-95.3153418809889, 39.5855661321772),
     (-95.3151845815691, 39.5854269561457),
     (-95.3149868196427, 39.5853419921505),
     (-95.3146319072194, 39.585349864589),
     (-95.3143160087592, 39.5854561090264),
     (-95.3138795781139, 39.5857580102494),
     (-95.3134781994929, 39.5862037717507),
     (-95.3132946141649, 39.5864811796879),
     (-95.3132383126639, 39.58684138

## Analyze Results

Let's examine the soil data we've collected. Each feature contains:
- **mukey**: Unique identifier for the soil map unit
- **area_acre**: Area of intersection with our study polygon (in acres)
- **geometry**: GeoJSON geometry of the intersecting area

In [12]:
# Examine a specific soil feature in detail
# This shows the complete data structure for one soil map unit intersection
print("Sample soil feature (third feature in the list):")
print("Structure: mukey, area_acres, area_square_meters, geometry")
processed_soil_features[2]

Sample soil feature (third feature in the list):
Structure: mukey, area_acres, area_square_meters, geometry


{'mukey': '2505122',
 'area_acres': np.float64(2.4359324227312196),
 'area_square_meters': np.float64(9857.868769462228),
 'geometry': {'type': 'Polygon',
  'coordinates': (((-95.311829380114, 39.5834043161128),
    (-95.3122979732325, 39.5834256319292),
    (-95.3123998466032, 39.5834618860383),
    (-95.3125382024731, 39.583615239146),
    (-95.3122789450899, 39.584066238992),
    (-95.3122863360114, 39.5841537305184),
    (-95.3124108086943, 39.5844078437466),
    (-95.3123855768292, 39.5845897457709),
    (-95.3119832651559, 39.5847633284736),
    (-95.31161516601344, 39.58486029043444),
    (-95.311611, 39.584283),
    (-95.31160275787104, 39.58344042963594),
    (-95.311829380114, 39.5834043161128)),)}}

### Additional Analysis: Combining Soil Properties with Spatial Data

Let's create a comprehensive analysis by combining our soil properties data with the spatial information:

In [14]:
# Combine spatial data with soil properties for comprehensive analysis
# Create a unified dataset with both geometric and soil property information

# Convert processed features to DataFrame for easier analysis
spatial_df = pd.DataFrame(processed_soil_features)

# Merge with soil properties data to get complete information
comprehensive_soil_analysis = pd.merge(
    spatial_df, 
    soil_properties_df, 
    on='mukey', 
    how='inner'
)

print("Comprehensive Soil Analysis Dataset:")
print(f"Shape: {comprehensive_soil_analysis.shape}")
print(f"Columns: {list(comprehensive_soil_analysis.columns)}")

# Calculate area-weighted statistics
total_area = comprehensive_soil_analysis['area_acres'].sum()
print(f"\nSpatial Summary:")
print(f"Total study area: {total_area:.3f} acres")
print(f"Number of distinct soil types: {len(comprehensive_soil_analysis)}")

# Show the most common soil characteristics
print("\nMost common soil characteristics in study area:")
if not comprehensive_soil_analysis.empty:
    largest_soil_unit = comprehensive_soil_analysis.loc[comprehensive_soil_analysis['area_acres'].idxmax()]
    print(f"Largest soil unit: {largest_soil_unit['muname']} ({largest_soil_unit['area_acres']:.3f} acres)")
    print(f"  - Taxonomic Order: {largest_soil_unit['taxorder']}")
    print(f"  - Drainage Class: {largest_soil_unit['drainagecl']}")
    print(f"  - Texture: {largest_soil_unit['texture']}")
    if pd.notna(largest_soil_unit['ph1to1h2o_r']):
        print(f"  - pH: {float(largest_soil_unit['ph1to1h2o_r']):.2f}")
    if pd.notna(largest_soil_unit['om_r']):
        print(f"  - Organic Matter: {float(largest_soil_unit['om_r']):.2f}%")

# Display the complete dataset
comprehensive_soil_analysis

Comprehensive Soil Analysis Dataset:
Shape: (3, 16)
Columns: ['mukey', 'area_acres', 'area_square_meters', 'geometry', 'muname', 'taxorder', 'taxsuborder', 'drainagecl', 'taxreaction', 'taxclname', 'ph1to1h2o_r', 'om_r', 'sandtotal_r', 'silttotal_r', 'claytotal_r', 'texture']

Spatial Summary:
Total study area: 97.338 acres
Number of distinct soil types: 3

Most common soil characteristics in study area:
Largest soil unit: Grundy silty clay loam, 1 to 3 percent slopes (56.961 acres)
  - Taxonomic Order: Mollisols
  - Drainage Class: Somewhat poorly drained
  - Texture: SICL
  - pH: 6.10
  - Organic Matter: 3.70%


,mukey,area_acres,area_square_meters,geometry,muname,taxorder,taxsuborder,drainagecl,taxreaction,taxclname,ph1to1h2o_r,om_r,sandtotal_r,silttotal_r,claytotal_r,texture
0,1475356,56.960623,230511.462895,"{'type': 'Polygon', 'coordinates': (((-95.3165...","Grundy silty clay loam, 1 to 3 percent slopes",Mollisols,Udolls,Somewhat poorly drained,not used,"Fine, smectitic, mesic Aquertic Argiudolls",6.1,3.7,4,65,31,SICL
1,1475358,37.941159,153542.422749,"{'type': 'MultiPolygon', 'coordinates': [(((-9...","Grundy silty clay loam, 3 to 7 percent slopes,...",Mollisols,Udolls,Somewhat poorly drained,not used,"Fine, smectitic, mesic Aquertic Argiudolls",6.1,3,3,65,32,SICL
2,2505122,2.435932,9857.868769,"{'type': 'Polygon', 'coordinates': (((-95.3118...","Pawnee clay loam, 4 to 8 percent slopes, eroded",Mollisols,Udolls,Moderately well drained,not used,"Fine, smectitic, mesic Oxyaquic Vertic Argiudolls",5.6,2,23,46,31,CL
